## Fast look to the data

In [67]:
import pandas as pd
import re

In [68]:
import pandas as pd
data = pd.read_csv("merged_cleaned.csv")
data.head()

,Id,date,platform,title,News content,Label
0,1,2023-01-11 00:00:00,Aljazeera,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,هدمت قوات الاحتلال الإسرائيلي -اليوم الأربعاء-...,real
1,2,2023-01-11 00:00:00,Aljazeera,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,خرجت مظاهرات في عدد من المدن الأوروبية مساء ال...,real
2,3,2023-01-11 00:00:00,Aljazeera,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,استشهد 4 فلسطينيين واعتقل عشرات آخرون -اليوم ا...,real
3,4,2023-02-11 00:00:00,Aljazeera,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,أكد الناطق باسمكتائب الشهيد عز الدين القسام-ال...,real
4,5,2023-03-11 00:00:00,Aljazeera,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات,استشهد 9 فلسطينيين في مواجهات اندلعت مع قوات ا...,real


In [69]:
num_missing_titles = data['News content'].isna().sum()
num_missing_titles

np.int64(0)

In [70]:
data['platform'].value_counts().head(4)

platform
Aljazeera    3422
Misbar       1426
Tibyan        247
X              64
Name: count, dtype: int64

In [71]:
data['Label'].value_counts()

Label
real    3913
fake    1439
Name: count, dtype: int64

## Merge title with news content

In [72]:
processedData= pd.DataFrame()

In [73]:
processedData['full_news'] = data['title'] + " " + data['News content']

## Map Label to 0 and 1

In [74]:
processedData['Label'] = data['Label'].map({'real': 1, 'fake': 0})
processedData.head()

,full_news,Label
0,الضفة الغربية.. الاحتلال يهدم 17 منزلا تاريخيا...,1
1,مظاهرات بمدن أوروبية تضامنا مع غزة وحشود أمام ...,1
2,شهداء في جنين وطولكرم وإضراب عام بالضفة الغربي...,1
3,أبو عبيدة: خسائر العدو أكبر بكثير مما يعلن وسن...,1
4,9 شهداء بالضفة والاحتلال يشن حملة اعتقالات است...,1


## Clean news content

In [75]:
import re

def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    if not isinstance(text, str):
        return ""

    # Remove URLs
    text = re.sub(r'http\S+|www.\S+|pic\.twitter\.com/\S+', '', text)


    # Remove HTML
    text = re.sub(r'<.*?>', '', text)


    # Remove Emojies 
    text = remove_emojis(text)


    # Remove Arabic diacritics
    text = re.sub(r'[\u064B-\u0652]', '', text)


    # # Remove punctuation (Arabic and general)
    # text = re.sub(r'[ـ،؛؟!:\.\,\(\)\[\]\{\}"\'«»\-_~…]', '', text)


    text = re.sub(r'[إأآا]', 'ا', text)

    lines = text.split('\n')


    cleaned_lines = []
    for line in lines:
        # Remove line if it not arabic
        if re.fullmatch(r'[^\u0600-\u06FF]+', line.strip()):
            continue


        # Keep Arabic lines 
        line = re.sub(r'[^\u0600-\u06FF\s]', '', line)


        # Remove empty lines
        if line.strip():
            cleaned_lines.append(line.strip())

    # Join the cleaned lines
    return '\n'.join(cleaned_lines)


In [76]:
processedData['full_news'] = processedData['full_news'].apply(clean_text)

## Remove Stop Words

In [77]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('arabic'))

In [78]:
processedData['full_news_no_stopwords'] = processedData['full_news'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in stop_words])
)

In [79]:
import requests

url = "https://raw.githubusercontent.com/mohataher/arabic-stop-words/master/list.txt"
response = requests.get(url)

with open("list.txt", "w", encoding="utf-8") as f:
    f.write(response.text)

with open("list.txt", encoding="utf-8") as f:
    arabic_stopwords = set(f.read().splitlines())



In [80]:
processedData['full_news_no_stopwords'] = processedData['full_news_no_stopwords'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in arabic_stopwords])
)



In [81]:
processedData.to_csv('ProcessedData_StopwordsStep.csv', index=False, encoding='utf-8-sig')


In [82]:
processedData.head()

,full_news,Label,full_news_no_stopwords
0,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومس...,1,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومسي...
1,مظاهرات بمدن اوروبية تضامنا مع غزة وحشود امام ...,1,مظاهرات بمدن اوروبية تضامنا غزة وحشود داونينغ ...
2,شهداء في جنين وطولكرم واضراب عام بالضفة الغربي...,1,شهداء جنين وطولكرم واضراب بالضفة الغربية تنديد...
3,ابو عبيدة خسائر العدو اكبر بكثير مما يعلن وسنج...,1,ابو عبيدة خسائر العدو اكبر بكثير يعلن وسنجعل غ...
4,شهداء بالضفة والاحتلال يشن حملة اعتقالات استشه...,1,شهداء بالضفة والاحتلال يشن حملة اعتقالات استشه...


In [83]:
from IPython.display import display, Markdown
test = 800

original_title = data.iloc[test]['title']
original_content = data.iloc[test]['News content']

cleaned = processedData.iloc[test]['full_news']
cleaned_no_stopwords = processedData.iloc[test]['full_news_no_stopwords']

display(Markdown("##  Original News Content"))
display(Markdown(f"**Title:** {original_title}"))
display(Markdown(f"**Length:** {len(original_title + ' ' + original_content)} characters"))
display(Markdown(original_content))

display(Markdown("##  Cleaned News"))
display(Markdown(f"**Length:** {len(cleaned)} characters"))
display(Markdown(cleaned))

display(Markdown("##  Cleaned News (No Stopwords)"))
display(Markdown(f"**Length:** {len(cleaned_no_stopwords)} characters"))
display(Markdown(cleaned_no_stopwords))


##  Original News Content

**Title:** "لا تتركونا نشيخ".. القسام تنشر رسالة من أسرى إسرائيليين لحكومتهم

**Length:** 1182 characters

نشرت كتائب القسام -الجناح العسكري لحركة المقاومة الإسلامية (حماس)- مقطعا مرئيا يظهر فيه 3 أسرى إسرائيليين في قطاع غزة يطالبون قادتهم بعدم تركهم في الأسر و"ألا يشيخوا فيه".

وقال أحد الأسرى الإسرائيليين ويُدعى حاييم بري (79 عاما)، إنه من كيبوتس نير عوز، وموجود في الأسر برفقة مجموعة كبار في السن، مرضى بأمراض مزمنة، ويعانون ظروفا قاسية للغاية.

وأضاف "نحن جيل بنى الدولة (إسرائيل)، ونحن شاركنا في بناء الجيش، ولا أفهم لماذا نحن متروكون هنا؟"، قبل أن يكمل مناشدا على ما يبدو رئيس الوزراء الإسرائيلي بنيامين نتنياهو، بالقول "يجب عليك أن تفرج عنا بكل ثمن".

وتابع "نحن لا نريد أن نكون ضحايا لاستهدافات سلاح الجو، أفرج عنا من دون أي شرط، ولا تتركونا نشيخ".

ونجحت وساطة قطرية في التوصل إلى هدنة إنسانية مؤقتة استمرت أسبوعا، وتم خلالها الإفراج عن محتجزين إسرائيليين في قطاع غزة، مقابل إطلاق سراح أسرى فلسطينيين من النساء والأطفال وفق معادلة (1 مقابل 3)، قبل أن تنهار الهدنة مطلع ديسمبر/كانون الأول الجاري.

وتتصاعد المظاهرات في تل أبيب المطالبة بضرورة إبرام صفقة تبادل جديدة خاصة بعد مقتل 3 أسرى إسرائيليين في غزة على يد قوات الاحتلال، وهو ما أثار موجة غضب عارمة وألقت مزيدا من الضغوط على حكومة نتنياهو اليمينية.

##  Cleaned News

**Length:** 1131 characters

لا تتركونا نشيخ القسام تنشر رسالة من اسرى اسرائيليين لحكومتهم نشرت كتائب القسام الجناح العسكري لحركة المقاومة الاسلامية حماس مقطعا مرئيا يظهر فيه  اسرى اسرائيليين في قطاع غزة يطالبون قادتهم بعدم تركهم في الاسر والا يشيخوا فيه
وقال احد الاسرى الاسرائيليين ويدعى حاييم بري  عاما، انه من كيبوتس نير عوز، وموجود في الاسر برفقة مجموعة كبار في السن، مرضى بامراض مزمنة، ويعانون ظروفا قاسية للغاية
واضاف نحن جيل بنى الدولة اسرائيل، ونحن شاركنا في بناء الجيش، ولا افهم لماذا نحن متروكون هنا؟، قبل ان يكمل مناشدا على ما يبدو رئيس الوزراء الاسرائيلي بنيامين نتنياهو، بالقول يجب عليك ان تفرج عنا بكل ثمن
وتابع نحن لا نريد ان نكون ضحايا لاستهدافات سلاح الجو، افرج عنا من دون اي شرط، ولا تتركونا نشيخ
ونجحت وساطة قطرية في التوصل الى هدنة انسانية مؤقتة استمرت اسبوعا، وتم خلالها الافراج عن محتجزين اسرائيليين في قطاع غزة، مقابل اطلاق سراح اسرى فلسطينيين من النساء والاطفال وفق معادلة  مقابل ، قبل ان تنهار الهدنة مطلع ديسمبركانون الاول الجاري
وتتصاعد المظاهرات في تل ابيب المطالبة بضرورة ابرام صفقة تبادل جديدة خاصة بعد مقتل  اسرى اسرائيليين في غزة على يد قوات الاحتلال، وهو ما اثار موجة غضب عارمة والقت مزيدا من الضغوط على حكومة نتنياهو اليمينية

##  Cleaned News (No Stopwords)

**Length:** 936 characters

تتركونا نشيخ القسام تنشر رسالة اسرى اسرائيليين لحكومتهم نشرت كتائب القسام الجناح العسكري لحركة المقاومة الاسلامية حماس مقطعا مرئيا يظهر اسرى اسرائيليين قطاع غزة يطالبون قادتهم بعدم تركهم الاسر والا يشيخوا الاسرى الاسرائيليين ويدعى حاييم بري عاما، كيبوتس نير عوز، وموجود الاسر برفقة مجموعة كبار السن، مرضى بامراض مزمنة، ويعانون ظروفا قاسية للغاية جيل بنى الدولة اسرائيل، ونحن شاركنا بناء الجيش، افهم متروكون هنا؟، يكمل مناشدا يبدو رئيس الوزراء الاسرائيلي بنيامين نتنياهو، بالقول يجب تفرج عنا بكل ثمن وتابع نريد نكون ضحايا لاستهدافات سلاح الجو، افرج عنا شرط، تتركونا نشيخ ونجحت وساطة قطرية التوصل هدنة انسانية مؤقتة استمرت اسبوعا، وتم خلالها الافراج محتجزين اسرائيليين قطاع غزة، اطلاق سراح اسرى فلسطينيين النساء والاطفال وفق معادلة تنهار الهدنة مطلع ديسمبركانون الجاري وتتصاعد المظاهرات تل ابيب المطالبة بضرورة ابرام صفقة تبادل جديدة مقتل اسرى اسرائيليين غزة يد قوات الاحتلال، اثار موجة غضب عارمة والقت مزيدا الضغوط حكومة نتنياهو اليمينية

## Remove NERs

In [84]:
# !pip install --upgrade torch numpy stanza
# !pip install numpy --upgrade
# import stanza
# stanza.download('ar')



In [85]:
# import pandas as pd
# import re
# import pandas as pd
# processedData = pd.read_csv("ProcessedData_StopwordsStep.csv")
# processedData.head()

In [86]:
# nlp = stanza.Pipeline(lang='ar',processors='tokenize,ner')

# def remove_named_entities_stanza(text):
#     doc = nlp(text)
#     ents = [ent.text for ent in doc.ents]
    
#     for ent in ents:
#         text = text.replace(ent, '')
    
#     return text.strip()

In [87]:
# processedData['full_news_no_ner'] = processedData['full_news_no_stopwords'].apply(remove_named_entities_stanza)



In [88]:
processedDataNoStopwordsNoNERs = pd.read_csv('processedDataNoStopwordsNoNERs.csv')
processedDataNoStopwordsNoNERs


,full_news,Label,full_news_no_stopwords,full_news_no_ner
0,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومس...,1,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومسي...,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومسي...
1,مظاهرات بمدن اوروبية تضامنا مع غزة وحشود امام ...,1,مظاهرات بمدن اوروبية تضامنا غزة وحشود داونينغ ...,مظاهرات بمدن تضامنا وحشود خرجت مظاهرات المد...
2,شهداء في جنين وطولكرم واضراب عام بالضفة الغربي...,1,شهداء جنين وطولكرم واضراب بالضفة الغربية تنديد...,شهداء واضراب تنديدا بمجازر استشهد واعتقل ...
3,ابو عبيدة خسائر العدو اكبر بكثير مما يعلن وسنج...,1,ابو عبيدة خسائر العدو اكبر بكثير يعلن وسنجعل غ...,خسائر العدو اكبر بكثير يعلن وسنجعل لعنة التار...
4,شهداء بالضفة والاحتلال يشن حملة اعتقالات استشه...,1,شهداء بالضفة والاحتلال يشن حملة اعتقالات استشه...,شهداء بالضفة والاحتلال يشن حملة اعتقالات استشه...
...,...,...,...,...
5346,الحرب على غزة مباشر المقاومة تكبد الاحتلال خسا...,1,الحرب غزة مباشر المقاومة تكبد الاحتلال خسائر ك...,الحرب مباشر المقاومة تكبد الاحتلال خسائر كبير...
5347,تصريح قديم لمقاتل من كتائب القسام حول اشتباكات...,0,تصريح قديم لمقاتل كتائب القسام اشتباكات قوات ا...,تصريح قديم لمقاتل اشتباكات الحرب الجارية تتد...
5348,الفيديو الذي يهدد فيه احمد موسى حماس قديم وليس...,0,الفيديو يهدد احمد موسى حماس قديم الحرب غزة تتد...,الفيديو يهدد قديم الحرب تتداول صفحات وحسابات...
5349,تدعم قطع الدعم عن الاونروا منظمات دولية تنشر ت...,1,تدعم قطع الدعم الاونروا منظمات دولية تنشر تقار...,تدعم قطع الدعم منظمات دولية تنشر تقارير دقيقة...


## Lemmatization 

In [89]:
# import stanza
# from tqdm import tqdm 

# stanza.download('ar')
# nlp = stanza.Pipeline(lang='ar', processors='tokenize,mwt,pos,lemma')
# tqdm.pandas()

# def lemmatize_text(text):
#     try:
#         doc = nlp(text)
#         lemmas = [word.lemma for sent in doc.sentences for word in sent.words]
#         return ' '.join(lemmas)
#     except:
#         return ''  
    
# processedDataNoStopwordsNoNERs['full_news_lemma'] = processedDataNoStopwordsNoNERs['full_news_no_ner'].progress_apply(lemmatize_text)
# processedDataNoStopwordsNoNERs.to_csv('processedDataLemma.csv', index=False, encoding='utf-8-sig')


In [90]:
# import re

# def remove_diacritics(text):
#     arabic_diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
#     return re.sub(arabic_diacritics, '', text)

# processedDataNoStopwordsNoNERs['full_news_lemma_clean'] = processedDataNoStopwordsNoNERs['full_news_lemma'].astype(str).apply(remove_diacritics)


In [91]:
processedDataLemma = pd.read_csv('processedDataLemma.csv')
processedDataLemma.head(2)


,full_news,Label,full_news_no_stopwords,full_news_no_ner,full_news_lemma,full_news_lemma_clean
0,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومس...,1,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومسي...,الضفة الغربية الاحتلال يهدم منزلا تاريخيا ومسي...,ضِفَّة غَربِيّ اِحتِلَال هَدَم مَنزِل تَارِيخِ...,ضفة غربي احتلال هدم منزل تاريخي و مسيرة ندد ب ...
1,مظاهرات بمدن اوروبية تضامنا مع غزة وحشود امام ...,1,مظاهرات بمدن اوروبية تضامنا غزة وحشود داونينغ ...,مظاهرات بمدن تضامنا وحشود خرجت مظاهرات المد...,مُظَاهَرَة بِ مَدِينَة أَضَام هُوَ وَ حَشد خَر...,مظاهرة ب مدينة أضام هو و حشد خرج مظاهرة مدينة ...


In [98]:
from IPython.display import display, Markdown
test = 70


cleaned = processedDataLemma.iloc[test]['full_news']
cleaned_no_stopwords = processedDataLemma.iloc[test]['full_news_no_stopwords']
cleaned_no_ner = processedDataLemma.iloc[test]['full_news_no_ner']
full_news_lemma = processedDataLemma.iloc[test]['full_news_lemma']
full_news_lemma_clean = processedDataLemma.iloc[test]['full_news_lemma_clean']



display(Markdown("##  Cleaned News"))
display(Markdown(f"**Length:** {len(cleaned)} characters"))
display(Markdown(cleaned))

display(Markdown("##  Cleaned News (No Stopwords)"))
display(Markdown(f"**Length:** {len(cleaned_no_stopwords)} characters"))
display(Markdown(cleaned_no_stopwords))


display(Markdown("##  Cleaned News (No Ners)"))
display(Markdown(f"**Length:** {len(cleaned_no_ner)} characters"))
display(Markdown(cleaned_no_ner))


display(Markdown("##  Cleaned News (Lemma)"))
display(Markdown(f"**Length:** {len(full_news_lemma)} characters"))
display(Markdown(full_news_lemma))

display(Markdown("##  Cleaned News (Lemma No Diacritics )"))
display(Markdown(f"**Length:** {len(full_news_lemma_clean)} characters"))
display(Markdown(full_news_lemma_clean))



##  Cleaned News

**Length:** 255 characters

الفيديو من المكسيك وليس من الاشتباكات الاخيرة بين المقاومة وقوات الاحتلال تتداول حسابات وصفحات على مواقع التواصل الاجتماعي، منذ قليل، مقطع فيديو ادعت انه من الاشتباكات العنيفة الاخيرة بين المقاومة وقوات الاحتلال في مستوطنات غلاف غزة بعد عملية طوفان الاقصى

##  Cleaned News (No Stopwords)

**Length:** 198 characters

...

**Length:** 185 characters

الفيديو اِشتِبَاك مُقَاوَمَة تَدَاوَل حِسَاب وَ صَفحَة مَوقِع تَوَاصُل اِجتِمَاعِيّ ، قَلِيل ، مَقطَع فيديو أَدعَى اِشتِبَاك عَنِيف مُقَاوَمَة مُستَوطَنَة غِلَاف عَمَلِيَّة طوفان أَقصَى

##  Cleaned News (Lemma No Diacritics )

**Length:** 132 characters

الفيديو اشتباك مقاومة تداول حساب و صفحة موقع تواصل اجتماعي ، قليل ، مقطع فيديو أدعى اشتباك عنيف مقاومة مستوطنة غلاف عملية طوفان أقصى

## Feature Extraction: TF-IDF

In [93]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


In [94]:
vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = vectorizer.fit_transform(processedDataLemma['full_news_lemma_clean'].astype(str))

tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=vectorizer.get_feature_names_out())


In [95]:
tfidf_df.shape

(5351, 5000)

In [96]:
tfidf_df.head(3)

,آب,آخر,آل,آلة,آمن,آن,آنذاك,آنس,آوى,أب,...,يوليوتموز,يوم,يومي,يون,يونس,يوني,يونيوحزيان,يونيوحزيران,يين,١٤
0,0.0,0.046916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.053399,0.039006,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.038795,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.055778,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
# (Additional) Top words for each new 
top_words_per_doc = tfidf_df.apply(lambda row: row.sort_values(ascending=False).head(3).index.tolist(), axis=1)
processedDataLemma['Top words'] = top_words_per_doc
print(processedDataLemma[['full_news_lemma_clean', 'Top words']].tail(10))


                                  full_news_lemma_clean              Top words
5341  حرب مباشر قسام بث مشهد جديد ل معركة و حراك متس...    [حراك, جديد, متفرق]
5342  الحيثيات متعلق ب مقطع فيديو أظهر نازح اهتفى ؟ ...     [مشهد, احتلال, هو]
5343  الفيديو اشتباك قديم نشر حساب ب عربي موقع ، أمس...  [عربي, الفيديو, مسعف]
5344  الفيديو قديم ل مقتل قناص إسرائيلي تداول حساب و...     [قناص, مقتل, ادعى]
5345  توسع مداهمة هو و اقتحم محور اقتحم إسرائيلي مدي...     [داهم, هو, اقتحام]
5346  حرب مباشر مقاومة تكبد احتلال خسارة كبير و خلاف...     [تكبد, خلاف, أوقع]
5347  تصريح قديم ل مقتل اشتباك حرب جاري تداول حساب و...   [مقتل, تحدث, اشتباك]
5348  الفيديو هدد قديم حرب تداول صفحة و حساب موقعي ت...        [هدد, حد, حديث]
5349  دعم قطعة دعم منظمة دولي نشر تقرير دقيق تقاطع ر...    [منظمة, مراقبة, هو]
5350  لصر خريطة و مصدر مفتوح مساعدة معرفة حي و مبنى ...    [ضرر, خريطة, متضرر]
